## Init DB

In [1]:
import sqlite3


# neue Datenbank erstellen
conn = sqlite3.connect('prod_data.db')
cursor = conn.cursor()


cursor.execute("drop table produktstammdaten")
cursor.execute("drop table auftraege")
cursor.execute("drop table fertigungsplaetze")
cursor.execute("drop table arbeitsplaene")
cursor.execute("drop table ffz")


# Produktstammdaten 
cursor.execute('''create table if not exists produktstammdaten (
        id varchar primary key unique,  
        bezeichnung text,
        dlz_plan integer,
        losgroesse integer,
        Bestand_RTL integer,
        Bestand_FTL integer)''') # weitere Spalten sind zu definieren

cursor.execute('''create table if not exists auftraege (
        Charge integer primary key autoincrement,
        id varchar,
        FOREIGN KEY (id) REFERENCES produktstammdaten(id))''') # weitere Spalten sind zu definieren

cursor.execute('''create table if not exists arbeitsplaene (
        id varchar,
        nr varchar,
        bmg varchar,
        t_plan integer,
        PRIMARY KEY (id, nr),
        FOREIGN KEY (id) REFERENCES produktstammdaten(id))''') # weitere Spalten sind zu definieren

cursor.execute('''create table if not exists fertigungsplaetze (
        id varchar primary key unique,
        name varchar,
        x_area_OL integer,
        y_area_OL integer,
        x_area_UR integer,
        y_area_UR integer,
        x_input integer,
        y_input integer,
        x_output integer,
        y_output integer)''') # weitere Spalten sind zu definieren

cursor.execute('''create table if not exists ffz (
        id varchar primary key unique,
        speed integer,
        akkulaufzeit integer)''') # weitere Spalten sind zu definieren


# Setze den Startwert von Charge auf 20000000
cursor.execute('INSERT INTO sqlite_sequence (name, seq) VALUES ("auftraege", 19999999)')

# Stammdaten pflegen
cursor.execute('INSERT INTO produktstammdaten (id, bezeichnung, dlz_plan, losgroesse, bestand_rtl, bestand_ftl) VALUES (?, ?, ?, ?, ?, ?)', ('A2', 'Deckel', '150', '50', '1000', '2000'))
cursor.execute('INSERT INTO produktstammdaten (id, bezeichnung, dlz_plan, losgroesse, bestand_rtl, bestand_ftl) VALUES (?, ?, ?, ?, ?, ?)', ('A7', 'Welle', '50', '200', '1000', '2000'))

# Arbeitspläne
cursor.execute('INSERT INTO arbeitsplaene (id, nr, bmg, t_plan) VALUES (?, ?, ?, ?)',
               ('A2', '10', 'SAE', '70'))
cursor.execute('INSERT INTO arbeitsplaene (id, nr, bmg, t_plan) VALUES (?, ?, ?, ?)', 
               ('A2', '20', 'DRH', '50'))
cursor.execute('INSERT INTO arbeitsplaene (id, nr, bmg, t_plan) VALUES (?, ?, ?, ?)', 
               ('A2', '30', 'QPR', '60'))
cursor.execute('INSERT INTO arbeitsplaene (id, nr, bmg, t_plan) VALUES (?, ?, ?, ?)', 
               ('A7', '10', 'SAE', '40'))
cursor.execute('INSERT INTO arbeitsplaene (id, nr, bmg, t_plan) VALUES (?, ?, ?, ?)', 
               ('A7', '20', 'DRH', '70'))
cursor.execute('INSERT INTO arbeitsplaene (id, nr, bmg, t_plan) VALUES (?, ?, ?, ?)', 
               ('A7', '30', 'QPR', '20'))

# Fertigungsplätze
cursor.executemany('insert into fertigungsplaetze (id, name, x_area_OL, y_area_OL, x_area_UR, y_area_UR, x_input, y_input, x_output, y_output) values (?,?,?,?,?,?,?,?,?,?)',
                [
                ('RTL', 'Rohteillager',  335,   20,     590,    170,    375,   50,     450,    150),
                ('P1', 'Zuschnitt',      590,   80,     650,    260,    600,   195,    600,    250),
                ('P2', 'Drehen',         370,   265,    535,    410,    500,   280,    400,    280),
                ])


# FFZ
cursor.execute('INSERT INTO ffz (id, speed, akkulaufzeit) VALUES (?, ?, ?)', ('F1', '10', '100'))
cursor.execute('INSERT INTO ffz (id, speed, akkulaufzeit) VALUES (?, ?, ?)', ('F2', '14', '150'))

# Änderungen speichern
conn.commit()

# Verbindung schließen
conn.close()

## Check DB

In [2]:
# Verbindung zur bestehenden Datenbank herstellen
conn = sqlite3.connect('prod_data.db')
cursor = conn.cursor()

# Abfrage, um alle Tabellen in der Datenbank anzuzeigen
cursor.execute('''
    SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';
''')

# Alle Tabellen auflisten
tables = cursor.fetchall()

# Für jede Tabelle den Inhalt anzeigen
for table in tables:
    table_name = table[0]
    print(f"Inhalt der Tabelle: {table_name}")
    
    # Inhalt der Tabelle abfragen
    cursor.execute(f'SELECT * FROM {table_name}')
    rows = cursor.fetchall()
    
    # Spaltennamen der Tabelle anzeigen
    column_names = [description[0] for description in cursor.description]
    print(column_names)  # Spaltennamen drucken
    
    # Inhalt der Tabelle Zeile für Zeile drucken
    for row in rows:
        print(row)
    print("\n" + "-"*50 + "\n")  # Trennlinie für bessere Lesbarkeit

# Verbindung schließen
conn.close()

Inhalt der Tabelle: produktstammdaten
['id', 'bezeichnung', 'dlz_plan', 'losgroesse', 'Bestand_RTL', 'Bestand_FTL']
('A2', 'Deckel', 150, 50, 1000, 2000)
('A7', 'Welle', 50, 200, 1000, 2000)

--------------------------------------------------

Inhalt der Tabelle: auftraege
['Charge', 'id']

--------------------------------------------------

Inhalt der Tabelle: arbeitsplaene
['id', 'nr', 'bmg', 't_plan']
('A2', '10', 'SAE', 70)
('A2', '20', 'DRH', 50)
('A2', '30', 'QPR', 60)
('A7', '10', 'SAE', 40)
('A7', '20', 'DRH', 70)
('A7', '30', 'QPR', 20)

--------------------------------------------------

Inhalt der Tabelle: fertigungsplaetze
['id', 'name', 'x_area_OL', 'y_area_OL', 'x_area_UR', 'y_area_UR', 'x_input', 'y_input', 'x_output', 'y_output']
('RTL', 'Rohteillager', 335, 20, 590, 170, 375, 50, 450, 150)
('P1', 'Zuschnitt', 590, 80, 650, 260, 600, 195, 600, 250)
('P2', 'Drehen', 370, 265, 535, 410, 500, 280, 400, 280)

--------------------------------------------------

Inhalt der Ta